In [1]:
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

print("✅ Keys loaded:")
print(f"OpenAI API Key: {openai_api_key[:5]}***")
print(f"Tavily API Key: {tavily_api_key[:5]}***")

openai_client = OpenAI(api_key=openai_api_key)

✅ Keys loaded:
OpenAI API Key: sk-pr***
Tavily API Key: tvly-***


In [2]:
def print_markdown(text: str):
    display(Markdown(text))

In [15]:
import json  # Import the json module for handling JSON data
from typing_extensions import TypedDict  # Import TypedDict for type hinting
from agents import function_tool
from pydantic import BaseModel

In [16]:
# Define a TypedDict for the expected parameters for the Tavily search function
# A TypedDict is like a blueprint for a dictionary in Python
# It tells Python exactly what keys the dictionary should have and what type of values go with each key.

class TavilySearchParams(TypedDict):
    query: str         # The search query string
    max_results: int   # The maximum number of results to return

In [17]:
# A TypedDict describes the expected keys and value types for a dict.
# Here: a search "query" (string) and "max_results" (int).

class TavilySearchParams(TypedDict):
    query: str
    max_results: int


# This decorator (from OpenAI Agents SDK) registers the function as a tool
# that an agent can call. 

@function_tool
def tavily_search(params: TavilySearchParams) -> str:

    # Tavily search endpoint
    url = "https://api.tavily.com/search"

    # Tell the API we're sending JSON
    headers = {"Content-Type": "application/json"}

    # Build the request body:
    # api_key: your Tavily API key (assumed defined elsewhere as tavily_api_key)
    # query: taken from the params dict
    # max_results: use provided value or default to 3 if missing
    payload = {
        "api_key": tavily_api_key,
        "query": params["query"],
        "max_results": params.get("max_results", 3),
    }

    # Send the POST request with JSON body and headers
    response = requests.post(url, json = payload, headers = headers)
    if response.status_code == 200:
        results = response.json().get("results", [])
        summary = "\n".join([f"- {r['title']}: {r['content']}" for r in results])
        return summary if summary else "No relevant results found."
    else:
        return f"Tavily API error: {response.status_code}"

print("✅ Tavily search tool ready.")

✅ Tavily search tool ready.


In [20]:
# Let's define our Researcher AI Agent

# Define a Pydantic model for the agent's output.
# This ensures the agent's final output will have a single field: "summary" (string).
# Note that this will be used in both the researcher and analyst agents
class AnalysisSummary(BaseModel):
    summary: str


# Create an AI agent called "Researcher"
researcher_agent = Agent(name = "Researcher",
                         instructions = """
## Context
You are a research agent with access to the Tavily search tool.

## Instruction
Given a user query, use the Tavily search tool to find relevant information and summarize the key findings.

## Input
- A research query from the user.

## Output
- A summary of key findings in a maximum of 5 bullet points.
""",
    model = "gpt-4.1-mini",
    tools = [tavily_search],
    output_type = AnalysisSummary)

print("✅ Researcher AI agent is now ready")

✅ Researcher AI agent is now ready


In [22]:
# Let's run the researcher AI agent
researcher_result = await Runner.run(researcher_agent, "Why is Labubu so popular and what are the rarest Labubu collectibles?")
researcher_result

RunResult(input='Why is Labubu so popular and what are the rarest Labubu collectibles?', new_items=[ToolCallItem(agent=Agent(name='Researcher', handoff_description=None, tools=[FunctionTool(name='tavily_search', description='', params_json_schema={'$defs': {'TavilySearchParams': {'properties': {'query': {'title': 'Query', 'type': 'string'}, 'max_results': {'title': 'Max Results', 'type': 'integer'}}, 'required': ['query', 'max_results'], 'title': 'TavilySearchParams', 'type': 'object', 'additionalProperties': False}}, 'properties': {'params': {'$ref': '#/$defs/TavilySearchParams'}}, 'required': ['params'], 'title': 'tavily_search_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x12288d4e0>, strict_json_schema=True, is_enabled=True)], mcp_servers=[], mcp_config={}, instructions='\n## Context\nYou are a research agent with access to the Tavily search tool.\n\n## Instruction\nGiven a

In [23]:
print_markdown(f"### 🤖 Agent’s Answer\n{researcher_result.final_output.summary}")

### 🤖 Agent’s Answer
- Labubu is a plush collectible toy created by Chinese artist Kasing Lung and popularized globally through a collaboration with the Chinese company Pop Mart starting in 2019.
- Its popularity comes from its cute, quirky design and the thrill of collecting, especially since Labubus are sold in blind boxes, making certain figures rare and exciting to find.
- The rarity and exclusivity of some Labubu editions, such as the secret Chestnut Cocoa Labubu, drive their high resale value, with rare figures fetching over $149 on platforms like eBay.
- Collectors and fans, including celebrities, have fueled the craze by valuing Labubus for their unique artistic appeal and limited availability.
- The hype around Labubu also involves careful attention to authentic versus fake versions, with scams involving counterfeit “Lafufu” toys circulating in the market.

In [24]:
# Let's define our Analyst AI Agent

analyst_agent = Agent(name = "Analyst",
                      instructions = """
## Context
You are an analyst who receives research notes generated by the research agent.

## Instruction
Given the research notes, analyze the content and extract key trends, risks, or insights.

## Input
- Research notes (summaries of findings from the research agent).

## Output
- A concise analysis (no more than 2 paragraphs) highlighting key trends, risks, or insights.
""",
    model = "gpt-4.1-mini",
    output_type = AnalysisSummary,
)

print("✅ Analyst AI agent us now ready")

✅ Analyst AI agent us now ready


In [27]:
# Let's run the researcher AI agent
researcher_result = await Runner.run(researcher_agent, "Electric vehicle batteries trends in 2025")
researcher_result

RunResult(input='Electric vehicle batteries trends in 2025', new_items=[ToolCallItem(agent=Agent(name='Researcher', handoff_description=None, tools=[FunctionTool(name='tavily_search', description='', params_json_schema={'$defs': {'TavilySearchParams': {'properties': {'query': {'title': 'Query', 'type': 'string'}, 'max_results': {'title': 'Max Results', 'type': 'integer'}}, 'required': ['query', 'max_results'], 'title': 'TavilySearchParams', 'type': 'object', 'additionalProperties': False}}, 'properties': {'params': {'$ref': '#/$defs/TavilySearchParams'}}, 'required': ['params'], 'title': 'tavily_search_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x12288d4e0>, strict_json_schema=True, is_enabled=True)], mcp_servers=[], mcp_config={}, instructions='\n## Context\nYou are a research agent with access to the Tavily search tool.\n\n## Instruction\nGiven a user query, use the Tavily 

In [28]:
analyst_result = await Runner.run(analyst_agent, researcher_result.final_output.summary)

In [29]:
print_markdown(f"### 🤖 Agent’s Answer\n{analyst_result.final_output.summary}")

### 🤖 Agent’s Answer
The global EV battery market is experiencing rapid growth and dynamic shifts driven by technological advances and regional policy changes. Innovations such as improved battery technologies enhancing range and affordability, alongside faster charging and vehicle-to-grid solutions, are key trends shaping 2025. Meanwhile, the rise of second-life EV batteries presents new opportunities, with increasing involvement from OEMs and repurposing firms signaling a maturing circular economy in the sector. The significant market share of lithium iron phosphate (LFP) batteries highlights a cost-effective shift in chemistry, supported by robust production capabilities in the US and imports from China.

On the supply side, the global battery manufacturing capacity surged nearly 30% in 2024 to over 3 TWh, outpacing current market demand by a factor of three. This considerable scaling indicates strong industry preparedness for accelerated EV adoption and battery storage growth in the coming decade. However, the complexity introduced by regional policy shifts and evolving technologies requires agile strategies from major players to maintain or improve market positioning.

In [30]:
# Define a Pydantic model for the agent's output
# This specifies exactly what fields the Writer agent should return

class FinalReport(BaseModel):
    short_summary: str  # A brief 2–3 sentence executive summary
    markdown_report: str    # A detailed report in Markdown format (at least 500 words)
    follow_up_questions: list[str]    # A list of 3–5 suggested follow-up research questions


# Create an AI agent called "Writer"
writer_agent = Agent(
    name = "Writer",
    instructions = """
You are a senior market analyst tasked with generating an executive-level research report.

## Context
You will receive:
- The original user query (the research question or topic of interest).
- Summaries and analyses generated by helper tools: 'researcher_agent' (for bullet-point research findings) and 'analyst_agent' (for key trends, risks, or insights).

You may call these helper tools as needed to gather additional information or clarify findings.

## Instructions
Your job is to synthesize all available information and produce the following outputs:
1. **Executive Summary:** Write a concise summary (2–3 sentences) highlighting the most important findings relevant to the original query.
2. **Detailed Markdown Report:** Compose a comprehensive, well-structured report in markdown format (at least 500 words) that covers key findings, context, implications, and supporting evidence.
3. **Follow-up Research Questions:** Suggest 3–5 thoughtful follow-up questions for further investigation.

## Input
- The original user query.
- Summaries and analyses from the helper tools.

## Output
Return a structured object with:
- `short_summary`: The executive summary.
- `markdown_report`: The detailed markdown report.
- `follow_up_questions`: A list of 3–5 follow-up research questions.
""",
    model = "gpt-4.1-mini",
    output_type = FinalReport,
)

print("✅ Writer AI Agent is now ready")

✅ Writer AI Agent is now ready


In [31]:
writer_agent = Agent(
    name = "Writer",
    instructions = """
You are a senior market analyst tasked with generating an executive-level research report **in French**.

## Context
You will receive:
- The original user query (the research question or topic of interest).
- Summaries and analyses generated by helper tools: 'researcher_agent' (for bullet-point research findings) and 'analyst_agent' (for key trends, risks, or insights).

You may call these helper tools as needed to gather additional information or clarify findings.

## Instructions
Your job is to synthesize all available information and produce the following outputs in **French**:
1. **Executive Summary:** Write a concise summary (2–3 sentences) in French highlighting the most important findings relevant to the original query.
2. **Detailed Markdown Report:** Compose a comprehensive, well-structured report in French in markdown format (at least 500 words) that covers key findings, context, implications, and supporting evidence.
3. **Follow-up Research Questions:** Suggest 3–5 thoughtful follow-up questions in French for further investigation.

## Input
- The original user query.
- Summaries and analyses from the helper tools.

## Output
Return a structured object with:
- `short_summary`: The executive summary in French.
- `markdown_report`: The detailed markdown report in French.
- `follow_up_questions`: A list of 3–5 follow-up research questions in French.
""",
    model = "gpt-4.1-mini",
    output_type = FinalReport)


In [32]:
# Create a session to store and share information between agents
session = SQLiteSession("research_agent_practice")

# Define an async function that takes the user's question and runs the agents
async def manager_run(user_query: str):
    # Show the user’s question in a nice markdown format
    print_markdown(f"**User's Request:** {user_query}")

    # Step 1: Run the researcher agent to gather information about the user's query
    researcher_result = await Runner.run(researcher_agent, user_query, session = session)
    research_summary = researcher_result.final_output.summary  # Get the short research summary

    # Step 2: Run the analyst agent to analyze the research findings
    analyst_result = await Runner.run(analyst_agent, research_summary, session = session)
    analysis_summary = analyst_result.final_output.summary  # Get the short analysis summary

    # Step 3: Prepare the combined input for the writer agent
    input_for_writer = (
        f"Original query: {user_query}\n"
        f"Research summary: {research_summary}\n"
        f"Analysis summary: {analysis_summary}"
    )

    # Step 4: Run the writer agent to create the final report
    writer_result = await Runner.run(writer_agent, input_for_writer, session = session)
    final_output: FinalReport = writer_result.final_output  # The final report from the writer

    # Step 5: Display the results in a nice, clear format
    print_markdown("---")
    print_markdown(f"### 📝 **Short Summary:**\n{final_output.short_summary}")  # Quick overview
    print_markdown("\n\n-----------------\n\n")
    print_markdown(f"### 📄 **Full Report (Markdown):**\n{final_output.markdown_report}")  # Detailed report
    print_markdown("\n\n-----------------\n\n")
    print_markdown(
        "### 🔍 **Follow-Up Questions:**\n- "
        + "\n- ".join(final_output.follow_up_questions)  # Any extra questions the user might explore
    )
    print_markdown("\n\n-----------------\n\n")

In [33]:
await manager_run("What is the public sentiment and expert reviews about the Tesla Cybertruck?")

**User's Request:** What is the public sentiment and expert reviews about the Tesla Cybertruck?

---

### 📝 **Short Summary:**
Le Tesla Cybertruck bénéficie d'avis consommateurs globalement positifs, avec 83 % des propriétaires le recommandant, particulièrement pour ses performances et son confort. Toutefois, les avis d'experts et du public sont plus nuancés, soulignant un design polarisant et des questions sur sa praticité, ce qui reflète une division entre admiration pour ses capacités et scepticisme quant à son esthétique et son positionnement dans la gamme Tesla.



-----------------



### 📄 **Full Report (Markdown):**
# Rapport sur le sentiment public et les critiques d'experts concernant le Tesla Cybertruck

## Introduction
Le Tesla Cybertruck, véhicule électrique utilitaire futuriste lancé par Tesla, suscite depuis son annonce des opinions très contrastées. Cette étude synthétise les retours des consommateurs, les critiques d'experts et le ressenti général sur les plateformes publiques.

## Retours des consommateurs
Selon Kelley Blue Book, 83 % des propriétaires de la version 2025 du Cybertruck le recommandent, témoignant d'une satisfaction globale notable. Les consommateurs mettent en avant :

- Une performance remarquable, avec une accélération impressionnante et des capacités tout-terrain solides.
- Un intérieur spacieux et durable, assurant un confort notable pour les passagers.

Ces points forts renforcent l'image du Cybertruck comme un véhicule performant et confortable pour ses utilisateurs.

## Avis d'experts
Les experts saluent le design unique et «cool» du Cybertruck, mais signalent également plusieurs défauts et controverses, parfois liées au produit lui-même ou à la personnalité publique d'Elon Musk, fondateur de Tesla. Certains commentaires suggèrent que si le camion n'est pas mauvais en soi, il est toutefois un produit Tesla atypique, voire problématique dans la cohérence de la gamme.

## Sentiment public
Sur des forums comme Reddit, les avis sont plus mitigés, voire négatifs sur certains aspects :

- La conception esthétique suscite un rejet chez une partie des utilisateurs, qualifiée de trop audacieuse ou inadaptée.
- La praticité, notamment la capacité et l’usage du plateau de chargement, est remise en question par ceux habitués aux camions traditionnels.

Ces critiques illustrent une fracture entre l’innovation stylistique et les attentes fonctionnelles plus classiques.

## Analyse synthétique
Le Cybertruck incarne une approche disruptive dans le segment des pickups électriques. Le véhicule impressionne par ses qualités techniques et de confort, ce qui plaît à une majorité d'acheteurs. Cependant, le design polarisant et des éléments perçus comme peu pratiques alimentent des critiques récurrentes. Cette controverse reflète un défi pour Tesla : proposer des produits innovants qui réussissent à concilier audace esthétique, utilité et adhésion large du public.

## Conclusion
Le Tesla Cybertruck est à la fois salué pour ses innovations et critiqué pour son style et son pragmatisme. Cette double perception en fait un véhicule emblématique d'une transition vers une nouvelle ère automobile électrique, mais également un sujet de débat intense sur l'avenir du design et de la fonctionnalité dans ce segment.

---

*Source des données : Kelley Blue Book, analyses d'experts automobiles, discussions communautaires en ligne.*



-----------------



### 🔍 **Follow-Up Questions:**
- Quels sont les principaux problèmes pratiques rencontrés par les utilisateurs du Cybertruck ?
- Comment le design du Cybertruck influence-t-il son acceptation sur différents marchés internationaux ?
- Quelles améliorations Tesla prévoit-elle pour les prochaines versions du Cybertruck ?
- Comment le Cybertruck se compare-t-il en termes de performances et de coût aux autres pickups électriques concurrents ?
- Quel est l'impact des controverses liées à Elon Musk sur la perception du Cybertruck par le public et les experts ?



-----------------



### Another Task 

    Run the pipeline again using the writer agent with French language report generation capability (from the previous practice opportunity).

    Create a new multi-agent team that performs creative advertising:
        Define a Creative_Director agent that brainstorms 3–5 ad ideas.
        Define a Strategist agent that selects the top 2 ideas and explains the reasoning.
        Define a Copywriter agent that writes tweets for each top campaign.
        Build a pipeline where the output of each agent is passed to the next, similar to the previous example.
        Try running the full pipeline on a sample prompt (e.g., "Launch campaign for a new eco-friendly water bottle in Bali").


In [34]:
session = SQLiteSession("creative_pipeline_practice")

class CreativeIdeas(BaseModel):
    ideas: list[str]

creative_director = Agent(name = "Creative_Director",
                          instructions = """
Context:
You are a creative director at an advertising agency.

Instruction:
Brainstorm 3–5 creative ad campaign ideas for the given product.

Input:
The product name or description.

Output:
A list of 3–5 creative campaign ideas (as bullet points or short phrases).
""",
    model = "gpt-4.1-mini",
    output_type = CreativeIdeas)


In [35]:
class SelectedCampaigns(BaseModel):
    top_campaigns: list[str]
    reasoning: str

strategist = Agent(name = "Strategist",
                   instructions = """
Context:
You are a marketing strategist evaluating campaign ideas.

Instruction:
From the provided list of campaign ideas, select the top 2 that are most promising and explain your reasoning.

Input:
A list of campaign ideas (one per line).

Output:
A list of the top 2 campaign ideas and a paragraph explaining why they stand out.
""",
    model = "gpt-4.1-mini",
    output_type = SelectedCampaigns,
)

In [36]:
class TweetCopy(BaseModel):
    tweets: list[str]

copywriter = Agent(name = "Copywriter",
                   instructions = """
Context:
You are a copywriter creating social media content.

Instruction:
For each of the top campaign ideas provided, write 2–3 engaging tweets that could be used to promote the campaign.

Input:
The selected top campaign ideas (one per line).

Output:
A list of 2–3 tweets for each campaign idea.
""",
    model = "gpt-4.1-mini",
    output_type = TweetCopy)

In [37]:
async def creative_pipeline(product_name: str):
    director_result = await Runner.run(creative_director, product_name, session = session)
    ideas = director_result.final_output.ideas

    strategist_input = "\n".join(ideas)
    strategist_result = await Runner.run(strategist, strategist_input, session = session)
    top_campaigns = strategist_result.final_output.top_campaigns

    tweets_input = "\n".join(top_campaigns)
    copywriter_result = await Runner.run(copywriter, tweets_input, session = session)
    tweets = copywriter_result.final_output.tweets

    print_markdown("### **Top Campaigns:**\n- " + "\n- ".join(top_campaigns))
    print_markdown("### **Tweets:**\n- " + "\n- ".join(tweets))


In [38]:
await creative_pipeline("Launch campaign for a new eco-friendly water bottle in Bali")

### **Top Campaigns:**
- Ocean Clean-Up Challenge
- Refresh Bali, Refresh Earth

### **Tweets:**
- 🌊 Join the #OceanCleanUpChallenge in Bali! Collect ocean waste, protect our beautiful beaches, and earn your very own eco-friendly water bottle. Let's make every sip count for the planet! 💧🌍 #Sustainability #EcoFriendly
- Be part of the movement that cleans our oceans and quenches your thirst sustainably. Grab your eco-bottle now and join the #OceanCleanUpChallenge—because a cleaner Bali starts with YOU! 🐢💙
- Celebrate nature with art! 🎨 Our 'Refresh Bali, Refresh Earth' campaign partners with local Balinese artists to bring you limited-edition eco-friendly water bottles inspired by Bali's breathtaking landscapes. Get yours before they're gone! 🍃✨ #EcoArt #SustainableStyle
- Sip sustainably and support Bali's vibrant art scene! Discover unique designs by Balinese artists on our eco-friendly bottles—a perfect blend of culture and conservation. Refresh Bali, refresh the earth! 🌺💧 #SupportLocal #GreenLiving